In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from config import Config

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image

2023-03-22 20:46:04.401210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
from config import Config

data_path = Config.data_path
out_path = Config.out_path

# Get data for text and images 

In [2]:
from src.preprocess import get_image_text_label_id

2023-03-22 20:51:43.725087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train, val, class_names = get_image_text_label_id()

Found 33583 files belonging to 21 classes.
Using 26867 files for training.


2023-03-22 20:51:47.589839: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-22 20:51:47.593807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-22 20:51:47.593928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-22 20:51:47.594305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Found 33583 files belonging to 21 classes.
Using 6716 files for validation.


In [4]:
n_classes = len(class_names)

In [5]:
# Train and Val sets

train_images = train['images'] / 255.0
val_images = val['images'] / 255.0

train_lables =  train['labels']
val_labels = val['labels']

train_text = train['text']
val_text = val['text']


# Image embeddings

In [29]:
def get_image_embeddings(images):
    """
    Loads a trained Keras CNN model and produces embeddings for the given numpy.ndarray of images.

    Args:
        images (numpy.ndarray): A numpy array of images for which embeddings are to be produced.

    Returns:
        embeddings (numpy.ndarray): A numpy array of embeddings for the given images.

    """
    # Load the saved Keras model from the 'cnn' folder.
    model = tf.keras.models.load_model(Config.model_path / 'cnn')

    # Define a new model that outputs embeddings for the given input images.
    embedding_model = tf.keras.models.Model(inputs=model.input, outputs=model.layers[-2].output)

    # Produce embeddings for the input images using the embedding model.
    embeddings = embedding_model.predict(images)

    return embeddings

In [32]:
train_image_embeddings = get_image_embeddings(train_images[0:100])

1/1 [==============================] - 0s 86ms/step


In [33]:
train_image_embeddings

array([[-1.7830307 , -0.17228794,  0.04804261, ..., -0.9839945 ,
        -0.48064113, -1.2190042 ],
       [-1.788084  , -0.1884675 ,  0.04804261, ..., -0.98718786,
        -0.48890972, -1.2072499 ],
       [-1.7786212 , -0.17712045,  0.04804261, ..., -0.9820397 ,
        -0.47511935, -1.2187872 ],
       ...,
       [-1.7837408 , -0.17538548,  0.04804261, ..., -0.9876039 ,
        -0.48491168, -1.217246  ],
       [-1.7853229 , -0.18212628,  0.04804261, ..., -0.987591  ,
        -0.48899412, -1.2102402 ],
       [-1.7825965 , -0.18086481,  0.04804261, ..., -0.9878912 ,
        -0.4895444 , -1.2093325 ]], dtype=float32)

# Text embeddings

In [8]:
import tensorflow_hub as hub
import tensorflow_text

In [9]:
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1')
encoder = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3')

In [23]:
val_text[0:5].to_numpy()

array(['TUNGSTEN SOLDER PICK WITH HANDLE Solder Pick for picking up molten solder when making jewelry',
       'Galt Toys Sponge Painting Contents: 4 x 22ml pots of paint, 8 sponge shapes, sponge roller, sponge brush, 4 stencils, 3 x A3 paper sheets, 3 cards and envelopes, 5 gift tags.',
       'Fiskars Deckle Paper Scissor Edger (12-92187897 ) Fiskars Classic Paper Edgers are making a come back in the marketplace.  Add decorative edges to paper with ease.',
       'Staedtler(R) GeoStudent Compass Student compass draws circles up to 8" in diameter. Design offers easy-to-handle design and sturdy, all-metal construction. Centerwheel offers extremely fine settings. Compass includes 2mm lead refills.',
       'T.S. Shure Woodburning Creations Kit Wood Burning Creations'],
      dtype=object)

In [29]:
def get_text_embeddings(text, trainable=False):
    """
    Loads BERT and produces embeddings for the given numpy.ndarray of text.

    Args:
        text (numpy.ndarray): A numpy array of text for which embeddings are to be produced.

    Returns:
        embeddings (numpy.ndarray): A numpy array of embeddings for the given text.

    """
    # Load the BERT preprocessor and model.
    preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1')
    encoder = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3')
    
    # Set the trainability of the base encoder.
    encoder.trainable = trainable

    input = preprocess(text)
    embeddings = encoder(input)["pooled_output"]

    return embeddings

In [32]:
get_text_embeddings(val_text[0:5].to_numpy())

TensorShape([5, 768])

In [31]:
get_text_embeddings(val_text[0:5].to_numpy(), trainable=True)

<tf.Tensor: shape=(5, 768), dtype=float32, numpy=
array([[-0.853229  , -0.4701246 , -0.9118326 , ..., -0.9242439 ,
        -0.6083519 ,  0.75898755],
       [-0.63061345, -0.48536488, -0.9875117 , ..., -0.8972233 ,
        -0.64699835,  0.68899375],
       [-0.80999315, -0.34002525, -0.9851131 , ..., -0.9558615 ,
        -0.572348  ,  0.5664147 ],
       [-0.8411222 , -0.4495623 , -0.9691123 , ..., -0.90381736,
        -0.6503354 ,  0.6637933 ],
       [-0.91697705, -0.6176196 , -0.98456764, ..., -0.927929  ,
        -0.71662486,  0.8328254 ]], dtype=float32)>

In [ ]:
import pickle
with open(out_path / "train_embeddings.pkl", "wb") as fOut:
    pickle.dump({'sentences': train_text[~train_text.isna()].tolist(), 'embeddings': pooled_output},fOut)

In [21]:
def project_embeddings(
    embeddings, num_projection_layers, projection_dims, dropout_rate
):
    projected_embeddings = layers.Dense(units=projection_dims)(embeddings)
    for _ in range(num_projection_layers):
        x = tf.nn.gelu(projected_embeddings)
        x = layers.Dense(projection_dims)(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Add()([projected_embeddings, x])
        projected_embeddings = layers.LayerNormalization()(x)
    return projected_embeddings


def create_text_encoder(preprocess, bert,
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Set the trainability of the base encoder.
    bert.trainable = trainable
    # Receive the text as inputs.
    inputs = layers.Input(shape=(), dtype=tf.string, name="text_input")
    # Preprocess the text.
    bert_inputs = preprocess(inputs)
    # Generate embeddings for the preprocessed text using the BERT model.
    embeddings = bert(bert_inputs)["pooled_output"]
    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the text encoder model.
    return keras.Model(inputs, outputs, name="text_encoder")

In [22]:
text_encoder = create_text_encoder(preprocess=preprocess, bert=encoder,
    num_projection_layers=1, projection_dims=256, dropout_rate=0.1
)

TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name='text_input'), name='text_input', description="created by layer 'text_input'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [19]:
def project_embeddings(
    embeddings, num_projection_layers, projection_dims, dropout_rate
):
    projected_embeddings = layers.Dense(units=projection_dims)(embeddings)
    for _ in range(num_projection_layers):
        x = tf.nn.gelu(projected_embeddings)
        x = layers.Dense(projection_dims)(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Add()([projected_embeddings, x])
        projected_embeddings = layers.LayerNormalization()(x)
    return projected_embeddings


def create_text_encoder(preprocess, encoder,
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Load the BERT preprocessing module.
    preprocess = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2",
        name="text_preprocessing",
    )
    # Load the pre-trained BERT model to be used as the base encoder.
    bert = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1",
        "bert",
    )
    # Set the trainability of the base encoder.
    bert.trainable = trainable
    # Receive the text as inputs.
    inputs = layers.Input(shape=(), dtype=tf.string, name="text_input")
    # Preprocess the text.
    bert_inputs = preprocess(inputs)
    # Generate embeddings for the preprocessed text using the BERT model.
    embeddings = bert(bert_inputs)["pooled_output"]
    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the text encoder model.
    return keras.Model(inputs, outputs, name="text_encoder")

In [20]:
text_encoder = create_text_encoder(
    num_projection_layers=1, projection_dims=256, dropout_rate=0.1
)

TypeError: Expected `trainable` argument to be a boolean, but got: bert

In [45]:
text_encoder.summary()

NameError: name 'text_encoder' is not defined

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Load the pre-trained model
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(Config.IMG_HEIGHT, Config.IMG_WIDTH, 3))

# Freeze the layers in the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [8]:
# Add a new output layer with the number of classes equal to your problem
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(n_classes, activation='softmax')(x)

In [9]:
# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])



In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 49, 49, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 49, 49, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [26]:
# Train the model
train_images = train['images'] / 255.0
val_images = val['images'] / 255.0

train_lables =  train['labels']
val_labels = val['labels']

tf.config.run_functions_eagerly(True)

model.fit(train_images, train_lables,
          epochs=Config.EPOCHS,
          validation_data=(val_images, val_labels))


Epoch 1/30


/Users/teresabono/miniconda3/envs/re-prods/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 1s 1s/step - loss: 2.4794 - accuracy: 0.2500 - val_loss: 3.5124 - val_accuracy: 0.0312
Epoch 2/30
1/1 [==============================] - 1s 1s/step - loss: 1.5384 - accuracy: 0.6562 - val_loss: 3.6256 - val_accuracy: 0.0312
Epoch 3/30
1/1 [==============================] - 1s 1s/step - loss: 1.0469 - accuracy: 0.7188 - val_loss: 3.7082 - val_accuracy: 0.0312
Epoch 4/30
1/1 [==============================] - 1s 1s/step - loss: 0.6893 - accuracy: 0.8750 - val_loss: 3.7868 - val_accuracy: 0.0625
Epoch 5/30
1/1 [==============================] - 1s 1s/step - loss: 0.4375 - accuracy: 0.9375 - val_loss: 3.8907 - val_accuracy: 0.0625
Epoch 6/30
1/1 [==============================] - 1s 1s/step - loss: 0.2819 - accuracy: 1.0000 - val_loss: 3.9999 - val_accuracy: 0.0938
Epoch 7/30
1/1 [==============================] - 1s 1s/step - loss: 0.1872 - accuracy: 1.0000 - val_loss: 4.0969 - val_accuracy: 0.0938
Epoch 8/30
1/1 [==============================] - 1s

In [ ]:
train_images = train['images'] / 255.0
val_images = val['images'] / 255.0

train_lables =  train['labels']
val_labels = val['labels']



In [28]:
from keras.layers import Input, Dense, Dropout, Flatten, concatenate
from keras.models import Model
from keras.optimizers import Adam
import tensorflow_hub as hub
import tensorflow as tf

# Load the pre-trained image model
image_model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5", trainable=False),
    Dropout(0.5),
    Dense(512, activation='relu')
])

# Load the pre-trained language model
text_model = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=False)

# Define the joint embedding model
image_input = Input(shape=(Config.IMG_HEIGHT, Config.IMG_WIDTH, 3))
text_input = Input(shape=(), dtype=tf.string)




AttributeError: module 'tensorflow.core.protobuf.error_codes_pb2' has no attribute '_CODE'

In [ ]:
# Train the model
model.fit([train_images, train_text], train_labels, batch_size=32, epochs=10, validation_data=([val_images, val_text], val_labels))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([test_images, test_text], test_labels, batch_size=32)
print('Test accuracy:', accuracy)

In [ ]:


# Load the pre-trained CLIP model
model, preprocess = clip.load('ViT-B/32', device='cpu')

# Freeze the model's parameters
for layer in model.layers:
    layer.trainable = False

# Define the new classification head
input_image = Input(shape=(224, 224, 3))
input_text = Input(shape=(1,))
image_features = model.layers[3](input_image)
text_features = model.layers[4](input_text)
features = Concatenate()([image_features, text_features])
logits = Dense(units=10, activation='softmax')(features)
classifier = Model(inputs=[input_image, input_text], outputs=logits)

# Define the loss function and optimizer
criterion = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = 

# Define a custom dataset for your image-text pairs
class ImageTextDataset(tf.keras.utils.Sequence):
    def __init__(self, images, texts, labels, batch_size):
        self.images = images
        self.texts = texts
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.labels) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_images = self.image[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_texts = self.texts[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images =  batch_images / 255.0
        batch_images = np.array(batch_images)
        batch_texts = np.array(batch_texts)

        return [batch_images, batch_texts], batch_labels

BATCH_SIZE = 32

# Create the training and validation datasets
train_dataset = ImageTextDataset(train_images, train_texts, train_labels, BATCH_SIZE)
val_dataset = ImageTextDataset(val_images, val_texts, val_labels, BATCH_SIZE)

# Train the model
classifier.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history = classifier.fit(train_dataset, epochs=Config.EPOCHS, validation_data=val_dataset)

# Plot the training and validation accuracy curves
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.legend()
plt.show()


In [ ]:


# Load the pre-trained CLIP model
model, preprocess = clip.load('ViT-B/32', device='cpu')

# Freeze the model's parameters
for layer in model.layers:
    layer.trainable = False

# Define the new classification head
input_image = Input(shape=(224, 224, 3))
input_text = Input(shape=(1,))
image_features = model.layers[3](input_image)
text_features = model.layers[4](input_text)
features = Concatenate()([image_features, text_features])
logits = Dense(units=10, activation='softmax')(features)
classifier = Model(inputs=[input_image, input_text], outputs=logits)

# Define the loss function and optimizer
criterion = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = Adam(lr=0.001)

# Define a custom dataset for your image-text pairs
class MyDataset(tf.keras.utils.Sequence):
    def __init__(self, image_paths, texts, labels, batch_size):
        self.image_paths = image_paths
        self.texts = texts
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.labels) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_image_paths = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_texts = self.texts[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = [preprocess(Image.open(image_path).convert('RGB')) for image_path in batch_image_paths]
        batch_images = np.array(batch_images)
        batch_texts = np.array(batch_texts)

        return [batch_images, batch_texts], batch_labels

# Prepare your image-text pairs
image_paths = [...] # list of image paths
texts = [...] # list of text descriptions
labels = [...] # list of labels

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
train_image_paths, val_image_paths, train_texts, val_texts, train_labels, val_labels = train_test_split(image_paths, texts, labels, test_size=0.2, random_state=42)

batch_size = 32

# Create the training and validation datasets
train_dataset = MyDataset(train_image_paths, train_texts, train_labels, batch_size)
val_dataset = MyDataset(val_image_paths, val_texts, val_labels, batch_size)

# Train the model
classifier.compile(optimizer=optimizer, loss=criterion, metrics=['accuracy'])
history = classifier.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Plot the training and validation accuracy curves
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.legend()
plt.show()
